# Advent of Code 2021 - Day 15

In [447]:
@JvmInline value class Chiton(val risk: Int)
typealias Cave = List<List<Chiton>>
typealias Index = Pair<Int, Int>
typealias Path = List<Index>

In [448]:
import java.io.File

val cave: Cave = File("Day15.input.txt")
    .bufferedReader()
    .readLines()
    .map { line -> line.map(Char::digitToInt).map(::Chiton) }

## Part 1

Find the path from start to finish.

In [449]:
import java.util.PriorityQueue

val Cave.height: Int
    get() = cave.size
val Cave.width: Int
    get() = cave.first().size

operator fun Chiton.plus(other: Chiton): Chiton = Chiton(risk + other.risk)
operator fun Chiton.plus(value: Int) = Chiton(risk + value)

fun Index.down(): Index = (first + 1) to second
fun Index.up(): Index = (first - 1) to second
fun Index.right(): Index = first to (second + 1)
fun Index.left(): Index = first to (second - 1)

fun Index.adjacents(graph: Graph): List<Index> = listOf(down(), up(), right(), left()).filter { 
    it.first >= 0 && it.second >= 0 && it.first < graph.height && it.second < graph.width
}

data class TotalChiton(val at: Index, val total: Chiton)
fun interface Traveral {
    fun traverse(graph: Graph, chiton: TotalChiton): List<TotalChiton>
}

class Graph(val cave: Cave, val scale: Int) {
    val height: Int = cave.height * scale
    val width: Int = cave.width * scale

    val start: Index = 0 to 0
    val end: Index = (width - 1) to (height - 1)

    fun walk(traversal: Traveral): Int {
        val processing = PriorityQueue<TotalChiton> { left, right -> left.total.risk - right.total.risk }.apply {
            add(TotalChiton(start, Chiton(0)))
        }
        val visited = mutableSetOf<Index>()

        while (processing.isNotEmpty()) {
            val current = processing.poll()
            
            if (current.at == end) return current.total.risk
            if (current.at !in visited) {
                visited.add(current.at)
                processing.addAll(traversal.traverse(this, current))
            }
        }
        throw IllegalStateException()
    }
}

Graph(cave, 1).walk { graph, prev ->
    prev.at.adjacents(graph).map {
        TotalChiton(it, prev.total + graph.cave[it.first][it.second])
    }
}.also { println("Part 1 - $it")}

Graph(cave, 5).walk { graph, prev ->
    prev.at.adjacents(graph).map {
        val dx = it.first / cave.width
        val dy = it.second / cave.height
        val index = (it.first % graph.cave.width) to (it.second % graph.cave.height)
        val wrappedRisk = 1 + ((graph.cave[index.first][index.second].risk + dy + dx) - 1) % 9

        TotalChiton(it, prev.total + wrappedRisk)
    }
}.let { println("Part 2 - $it")}

Part 1 - 423
Part 2 - 2778


### Notes

This was a miserable problem for me. Started out using the grid, using recursion, using a graph, iterative approach, different semantics, etc. etc. etc.

This is "just" a pathfinding problem with weighted edges. There are many pathfinding solutions out there, this one tries to use Dijkstra's. The idea here is to spread out one cell at a time while keeping track of previous paths. The next path to spread out from is always the least total weighted path. You'll never have to move into a cell that had been visited by a previous path because there will always be a shorter path to get to that visited cell. This means a lot of backtracking in an unweighted graph, so it's important that this was weighted. In an unweighted / uniform graph, it would be better to use something like a depth-first search.

## Part 2

The cave repeats with a predictable pattern.

### Notes

Jupyter wouldn't let me put the code for part 2 in another cell, so it's above in part 1.